In [1]:
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, IntegerType, DateType, LongType
from pyspark.context import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import col

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1653653285629_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc = SparkSession
spark = SparkSession.builder.appName("recencia").getOrCreate()
sparkContext=spark.sparkContext

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#dataset sem gramatura
df = spark\
        .read.format("csv")\
        .option("inferSchema", "True")\
        .option("header", "True")\
        .csv("s3://bckt-crm-in-prd/IN_DATA_ZIP_ENCRYPTED/*")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
from pyspark.sql.functions import to_date


df1 = df.withColumn('data',to_date(df.data_emissao)).drop("data_emissao")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Codigo Stack Overflow

In [5]:
data_frame = df1.groupBy(["cod_item", "cpf"])\
.agg( 
  reverse( # collect all the dates in an array in descending order from most recent to oldest.
    array_sort(
      collect_list( df1.data ))).alias("data") )\
.select( 
  col("cpf"),col('cod_item'), 
  col("data")[0].alias("last_purchase"), # get last purchase
  col("data")[1].alias("penultimate")   ) #get last purchase + 1

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Dados Agrupados Item

In [6]:
new_data = data_frame.na.drop(subset=["penultimate"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
new_data.count()

In [ ]:
new_data.show(10)

In [7]:
#Date diff between cod_item 
date_dif = new_data.select(col('cod_item'), col('cpf'),
      col("last_purchase"),
      col("penultimate"),
      datediff(col("last_purchase"),col("penultimate")).alias("datediff"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
date_dif.count() / df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.10930866504548653

In [9]:
date_dif.coalesce(1).write.option("header","true").format("parquet").mode("overwrite").save("s3://bckt-crm-in-prd/row/table.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
date_dif.show(10)

In [ ]:
date_dif = date_dif.filter(date_dif.datediff != 0)

In [ ]:
date_dif.groupBy("cod_item").avg("datediff").show(20)